In [1]:
import csv
import os
import glob
import re
from pandas import DataFrame, Series
from openslide import open_slide
from PIL import Image
import timeit
import time
import math
import numpy as np
from scipy.ndimage.morphology import binary_fill_holes
from skimage.color import rgb2gray
from skimage.feature import canny
from skimage.morphology import binary_closing, binary_dilation, disk

In [2]:
train_paths = ["/scratch/ss4yd/chrc_data/train/Celiac/"]


images = {}
images_by_folder = {}
for train_path in train_paths:
    images_by_folder[str(train_path)] = []
    files = glob.glob(os.path.join(train_path, '*.svs'))
    for fl in files:
        flbase = os.path.basename(fl)
        flbase_noext = os.path.splitext(flbase)[0]
        images[flbase_noext]=fl
        images_by_folder[str(train_path)].append(flbase)

In [3]:
path_change_map = {}

for key in list(images_by_folder.keys()):
    temp = key.replace('chrc_data', 'chrc_data_patches')
    path_change_map[key] = temp

In [4]:
def create_patches(slide, patch_size, overlap):
    # steps to advance per axis with overlap
    step_size = patch_size - overlap
    
    # get dimensions of the image
    xlim = op_slide_img.level_dimensions[0][0]
    ylim = op_slide_img.level_dimensions[0][1]
    print("Dimensions x: " + str(xlim) + " y: " + str(ylim))
    
    # get the number of times to traverse each axis
    x_ind = math.ceil(xlim/(step_size))
    y_ind = math.ceil(ylim/(step_size))
    
    # initialize list to store patches
    patches = []
    
    # pixels left to traverse in the y-axis at the beginning of the traversal
    img_y_left = ylim
    # initialize the starting y corner
    y = 0 - step_size
    for y_ in range(y_ind):
        
        # initialize the starting x corner
        x = 0-step_size
        
        # advance the y axis (note: it starts with 0)
        y = y + step_size
        
        # pixels left to traverse in the x-axis at the beginning of the traversal
        img_x_left = xlim
        
        # update the number of pixels left to traverse
        img_y_left = img_y_left - step_size
        # print('y - left: ' + str(img_y_left))
        
        # if more than patch size left, get the (patch_size x patch_size) image
        if (img_y_left > 0 and img_y_left > patch_size):
            for x_ in range(x_ind):
                x = x + (step_size)
                img_x_left = img_x_left - step_size
                print(img_x_left)
                if (img_x_left > 0 and img_x_left > patch_size):
                    patches.append(op_slide_img.read_region(
                        (x, y), 0, (patch_size, patch_size)))
                elif (img_x_left < patch_size and img_x_left > 0):
                    x = xlim - patch_size
                    patches.append(op_slide_img.read_region((x, y), 0, (patch_size, patch_size)))
                    break
        # if less than patch size left, get the rest of the image, regardless of the overlap
        elif (img_y_left > 0 and img_y_left < patch_size):
            y = ylim - patch_size
            for x_ in range(x_ind):
                x = x + (step_size)
                img_x_left = img_x_left - step_size
                print(img_x_left)
                if (img_x_left > 0 and img_x_left > patch_size):
                    patches.append(op_slide_img.read_region(
                        (x, y), 0, (patch_size, patch_size)))
                elif (img_x_left < patch_size and img_x_left > 0):
                    x = xlim - patch_size
                    patches.append(op_slide_img.read_region(
                        (x, y), 0, (patch_size, patch_size)))
                    break
            break
    